# 4. Aggregation Pipelines: Let the Server Do It For You

You've used projection, sorting, indexing, and limits to speed up data fetching. But there are still annoying performance bottlenecks in your analysis pipelines. You still need to fetch a ton of data. Thus, network bandwidth and downstream processing and memory capacity still impact performance. This chapter is about using MongoDB to perform aggregations for you on the server.

In [1]:
# Importing libraries
import expectexception

from pymongo import MongoClient
from pprint import pprint

from collections import OrderedDict
from itertools import groupby
from operator import itemgetter
from pymongo.errors import OperationFailure

In [2]:
# Instantiating the mongodb client
client = MongoClient()

# Create local "nobel" database on the fly
db = client["nobel"]

## 4.1 Intro to Aggregation

### Queries have implicit stages

In [3]:
# Using find method
cursor = db.laureates.find(
    filter={"bornCountry": "USA"},
    projection={"prizes.year": 1},
    limit=3
)
for doc in cursor:
    print(doc["prizes"])

[{'year': '1972'}]
[{'year': '1972'}]
[{'year': '1975'}]


In [4]:
# Using aggregation method
cursor = db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}},
    {"$project": {"prizes.year": 1}},
    {"$limit": 3}
])
for doc in cursor:
    print(doc["prizes"])

[{'year': '1972'}]
[{'year': '1972'}]
[{'year': '1975'}]


### Adding sort and skip stages

In [5]:
list(db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}},
    {"$project": {"prizes.year": 1, "_id": 0}},
    {"$sort": OrderedDict([("prizes.year", 1)])},
    {"$skip": 1},
    {"$limit": 3}
]))

[{'prizes': [{'year': '1912'}]},
 {'prizes': [{'year': '1914'}]},
 {'prizes': [{'year': '1919'}]}]

### But can I count?

In [6]:
# Using agregations
list(db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}},
    {"$count": "n_USA-born-laureates"}
]))

[{'n_USA-born-laureates': 269}]

In [7]:
# Using count documents
db.laureates.count_documents({"bornCountry": "USA"})

269

### Excercises
#### Sequencing stages

In [8]:
# Here is a cursor, followed by four aggregation pipeline stages
cursor = (db.laureates.find(projection={"firstname": 1, 
                                        "prizes.year": 1, 
                                        "_id": 0},
                            filter={"gender": "org"})
                      .limit(3)
                      .sort("prizes.year", -1))

# What sequence pipeline of the above four stages can produce a 
# cursor db.laureates.aggregate(pipeline) equivalent to cursor above?
project_stage = {"$project": {"firstname": 1, "prizes.year": 1, "_id": 0}}
match_stage = {"$match": {"gender": "org"}}
limit_stage = {"$limit": 3}
sort_stage = {"$sort": {"prizes.year": -1}}

cursor_agg = db.laureates.aggregate([match_stage, project_stage, sort_stage, limit_stage])
assert list(cursor) == list(cursor_agg)

#### Aggregating a few individuals' country data

In [9]:
# Translate the below cursor cursor to an equivalent aggregation cursor, 
cursor = (db.laureates.find(
    filter={"gender": {"$ne": "org"}},
    projection=["bornCountry", "prizes.affiliations.country"]
).limit(3))

pipeline = [
    {'$match': {'gender': {"$ne": "org"}}},
    {'$project': {'bornCountry': 1, 'prizes.affiliations.country': 1}},
    {'$limit': 3}
]
cursor_agg = db.laureates.aggregate(pipeline)
assert list(cursor) == list(cursor_agg)

#### Passing the aggregation baton to Python

Construct an aggregation pipeline to collect, in reverse chronological order (i.e., descending year), prize documents for all original categories (that is, $in categories awarded in 1901). Project only the prize year and category (including document _id is fine).

The aggregation cursor will be fed to Python's itertools.groupby function to group prizes by year. For each year that at least one of the original prize categories was missing, a line with all missing categories for that year will be printed.

In [10]:
# Getting categories in 1901
original_categories = set(db.prizes.distinct("category", {"year": "1901"}))

# Save an pipeline to collect original-category prizes
pipeline = [
    {'$match': {'category': {'$in': list(original_categories)}}},
    {'$project': {'year': 1, 'category': 1}},
    {'$sort': OrderedDict([('year', -1)])}   # To preserve the order in the dictionary
]
cursor = db.prizes.aggregate(pipeline)
for key, group in groupby(cursor, key=itemgetter("year")):
    missing = original_categories - {doc["category"] for doc in group}
    if missing:
        print("{year}: {missing}".format(year=key, missing=", ".join(sorted(missing))))

2018: literature
1972: peace
1967: peace
1966: peace
1956: peace
1955: peace
1948: peace
1943: literature, peace
1939: peace
1935: literature
1934: physics
1933: chemistry
1932: peace
1931: physics
1928: peace
1925: medicine
1924: chemistry, peace
1923: peace
1921: medicine
1919: chemistry
1918: literature, medicine, peace
1917: chemistry, medicine
1916: chemistry, medicine, peace, physics
1915: medicine, peace
1914: literature, peace


## 4.2 Aggregation Operators and Grouping

### Field paths

In [11]:
# reviewing the collection
pprint(db.laureates.find_one({}))

{'_id': ObjectId('6706d88371ea025ecc350f6e'),
 'born': '1853-07-18',
 'bornCity': 'Arnhem',
 'bornCountry': 'the Netherlands',
 'bornCountryCode': 'NL',
 'died': '1928-02-04',
 'diedCountry': 'the Netherlands',
 'diedCountryCode': 'NL',
 'firstname': 'Hendrik Antoon',
 'gender': 'male',
 'id': '2',
 'prizes': [{'affiliations': [{'city': 'Leiden',
                               'country': 'the Netherlands',
                               'name': 'Leiden University'}],
             'category': 'physics',
             'motivation': '"in recognition of the extraordinary service they '
                           'rendered by their researches into the influence of '
                           'magnetism upon radiation phenomena"',
             'share': '2',
             'year': '1902'}],
 'surname': 'Lorentz'}


In [12]:
db.laureates.count_documents({'prizes.1': {'$exists': True}})

6

In [13]:
# Using expression object
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"prizes.share": 1}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'),
 'prizes': [{'share': '1'}, {'share': '1'}]}

In [14]:
# Counting prizes per laureate
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"n_prizes": {"$size": "$prizes"}}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'), 'n_prizes': 2}

In [15]:
# Renaming prizes.share to prize_share
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"prizes_share": "$prizes.share"}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'), 'prizes_share': ['1', '1']}

In [16]:
# Renaming surname to lastname
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"last_name": "$surname", "firstname": 1}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'),
 'firstname': 'Linus Carl',
 'last_name': 'Pauling'}

### Operator expressions

In [17]:
# the operator expression, which treats an operator as a function
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"n_prizes": {"$size": "$prizes"}}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'), 'n_prizes': 2}

In [18]:
# We could also write the operator expression as taking 
# a list of one element, and we get the same result. 
# For convenience, when an operator only has one parameter, 
# we can omit the brackets as above.
db.laureates.aggregate([
    {"$match": {'prizes.1': {'$exists': True}}},
    {"$project": {"n_prizes": {"$size": ["$prizes"]}}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350fab'), 'n_prizes': 2}

### Multi-parameter operator

In [19]:
# For example, here I use the dollar-in operator, 
# which takes two parameters. To get the array of 
# prize shares for a laureate, I use a field path. 
# I then project a new field, "solo winner", which 
# is true if and only if the array of prize shares 
# contains the string value "1".
db.laureates.aggregate([
    {"$project": {"solo_winner": {"$in": ['1', "$prizes.share"]},
                  "n_prizes": {"$size": "$prizes"},
                  "prizes.share": 1}}
]).next()

{'_id': ObjectId('6706d88371ea025ecc350f6e'),
 'prizes': [{'share': '2'}],
 'solo_winner': False,
 'n_prizes': 1}

### Implementing .distinct()

A group stage takes an expression object that must map the underscore-id field. As for any MongoDB document, the underscore-id field must be unique. In this case, each output document will have as its id a distinct value of the bornCountry field. All bornCountry values get captured because no match stage precedes the group stage. Thus, our list comprehension collecting id values collects all distinct bornCountry values. This includes the value None, which happens when a field is not present in a document.

In [20]:
# Using distinct
list_1 = set(db.laureates.distinct("bornCountry"))
print(len(list_1))  # Include None

# Using $group
list_2 = [doc["_id"] 
          for doc in db.laureates.aggregate([
             {"$group": {"_id": "$bornCountry"}}
         ])]
print(len(list_2))  # Include None

print(set(list_2) == set(list_1))

121
121
True


In [21]:
# Reviewing the created lists
pprint(list(list_1)[:5])
pprint(list(db.laureates.aggregate([{"$group": {"_id": "$bornCountry"}}]))[:5])

['Germany (now Russia)',
 'British India (now Bangladesh)',
 'Nigeria',
 'Prussia (now Russia)',
 'Russia']
[{'_id': None},
 {'_id': 'Argentina'},
 {'_id': 'Australia'},
 {'_id': 'Austria'},
 {'_id': 'Austria-Hungary (now Austria)'}]


### How many prizes have been awarded in total?

In [22]:
# A group stage takes an expression object that must map the underscore-id field
list(db.laureates.aggregate([
    {"$project": {"n_prizes": {"$size": "$prizes"}}},
    {"$group": {"_id": None, "n_prizes_total": {"$sum": "$n_prizes"}}}
]))

[{'_id': None, 'n_prizes_total': 941}]

In [23]:
%%expect_exception OperationFailure   

# a group specification must include an _id
list(db.laureates.aggregate([
    {"$project": {"n_prizes": {"$size": "$prizes"}}},
    {"$group": {"n_prizes_total": {"$sum": "$n_prizes"}}}
]))

---------------------------------------------------------------------------
OperationFailure                          Traceback (most recent call last)
Cell In[23], line 2
      1 # a group specification must include an _id
----> 2 list(db.laureates.aggregate([
      3     {"$project": {"n_prizes": {"$size": "$prizes"}}},
      4     {"$group": {"n_prizes_total": {"$sum": "$n_prizes"}}}
      5 ]))

File C:\ProgramData\anaconda3\envs\mongo\Lib\site-packages\pymongo\collection.py:2696, in Collection.aggregate(self, pipeline, session, let, comment, **kwargs)
   2620 """Perform an aggregation using the aggregation framework on this
   2621 collection.
   2622 
   (...)
   2693     https://mongodb.com/docs/manual/reference/command/aggregate
   2694 """
   2695 with self.__database.client._tmp_session(session, close=False) as s:
-> 2696     return self._aggregate(
   2697         _CollectionAggregationCommand,
   2698         pipeline,
   2699         CommandCursor,
   2700         session=

### Excercises
#### Field Paths and Sets

In [24]:
# Previously, we confirmed -- via a Python loop -- 
# that for each prize, either all laureates have a 1/3 share, 
# or none do. Now, let's do this via an aggregation 
# (result should be an empty list):
for doc in db.prizes.find({}, ["laureates.share"]):
    share_is_three = [laureate["share"] == "3" 
                      for laureate in doc["laureates"]]
assert all(share_is_three) or not any(share_is_three)
print(share_is_three)

[False, False]


In [25]:
# Solution Via an aggregation
list(db.prizes.aggregate([
    {"$project": {"allThree": {"$setEquals": ['$laureates.share', ['3']]},
                  "noneThree": {"$not": {"$setIsSubset": [['3'], '$laureates.share']}}}},
    {"$match": {"$nor": [{"allThree": True}, {"noneThree": True}]}}]))

[]

In [26]:
# Reviewing the data
db.prizes.find_one({})

{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'year': '2018',
 'category': 'physics',
 'overallMotivation': '“for groundbreaking inventions in the field of laser physics”',
 'laureates': [{'id': '960',
   'firstname': 'Arthur',
   'surname': 'Ashkin',
   'motivation': '"for the optical tweezers and their application to biological systems"',
   'share': '2'},
  {'id': '961',
   'firstname': 'Gérard',
   'surname': 'Mourou',
   'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
   'share': '4'},
  {'id': '962',
   'firstname': 'Donna',
   'surname': 'Strickland',
   'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
   'share': '4'}]}

In [27]:
# Building step by step in a different scenario
print('projection: laureates.share')
pprint(db.prizes.aggregate([{"$project": {'laureates.share': 1}}]).next())
print()

# $setEquals
print("Adding $setEquals ['2']")
pprint(db.prizes.aggregate([{
    "$project": {"allThree": {"$setEquals": ['$laureates.share', ['2']]},
                 'laureates.share': 1,
                 'share_items': '$laureates.share'}}
]).next())
print()

print("Adding $setEquals ['2', '4', '4']")
pprint(db.prizes.aggregate([{
    "$project": {"allThree": {"$setEquals": ['$laureates.share', ['2', '4', '4']]},
                 'laureates.share': 1,
                 'share_items': '$laureates.share'}}
]).next())
print()

print("Adding $setEquals ['2'] when there is only 2 laureates")
pprint(db.prizes.aggregate([
    {"$match": {'laureates.2': {'$exists': False}}},
    {"$project": {"allThree": {"$setEquals": ['$laureates.share', ['2']]},
                  'laureates.share': 1,
                  'share_items': '$laureates.share'}}
]).next())
print()

print("Adding '$setIsSubset'")
# Takes two arrays and returns true when the first array is a subset of the second
pprint(db.prizes.aggregate([
    {"$match": {'laureates.2': {'$exists': False}}},
    {"$project": {"allThree": {"$setEquals": ['$laureates.share', ['2']]},
                  'laureates.share': 1,
                  'share_items': '$laureates.share',
                  'is_subset': {'$setIsSubset': [['2'], '$laureates.share']}}},
]).next())

print("Adding '$not'")
pprint(db.prizes.aggregate([
    {"$match": {'laureates.2': {'$exists': False}}},
    {"$project": {"allThree": {"$setEquals": ['$laureates.share', ['2']]},
                  'laureates.share': 1,
                  'share_items': '$laureates.share',
                  'is_subset': {'$setIsSubset': [['2'], '$laureates.share']},
                  'not_is_subset': {'$not': {'$setIsSubset': [['2'], '$laureates.share']}}}},
]).next())

projection: laureates.share
{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'laureates': [{'share': '2'}, {'share': '4'}, {'share': '4'}]}

Adding $setEquals ['2']
{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'allThree': False,
 'laureates': [{'share': '2'}, {'share': '4'}, {'share': '4'}],
 'share_items': ['2', '4', '4']}

Adding $setEquals ['2', '4', '4']
{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'allThree': True,
 'laureates': [{'share': '2'}, {'share': '4'}, {'share': '4'}],
 'share_items': ['2', '4', '4']}

Adding $setEquals ['2'] when there is only 2 laureates
{'_id': ObjectId('6706d88371ea025ecc350d21'),
 'allThree': True,
 'laureates': [{'share': '2'}, {'share': '2'}],
 'share_items': ['2', '2']}

Adding '$setIsSubset'
{'_id': ObjectId('6706d88371ea025ecc350d21'),
 'allThree': True,
 'is_subset': True,
 'laureates': [{'share': '2'}, {'share': '2'}],
 'share_items': ['2', '2']}
Adding '$not'
{'_id': ObjectId('6706d88371ea025ecc350d21'),
 'allThree': True,
 'is_subset': Tru

#### Organizing prizes

In [28]:
# How many prizes were awarded (at least partly) to organizations?
pipeline = [
    {'$match': {'gender': "org"}},
    {"$project": {"n_prizes": {"$size": '$prizes'}}},
    {"$group": {"_id": None, "n_prizes_total": {"$sum": '$n_prizes'}}}
]

print(list(db.laureates.aggregate(pipeline)))

[{'_id': None, 'n_prizes_total': 27}]


#### Gap years, aggregated

In a previous exercise, you collected instances of prize categories not being awarded in particular years. You implemented this using a for loop in Python. You will now implement this as an aggregation pipeline that:

- Filters for original prize categories (i.e. sans economics),
- Projects category and year,
- Groups distinct prize categories awarded by year,
- Projects prize categories not awarded by year,
- Filters for years with missing prize categories, and
- Returns a cursor of documents in reverse chronological order, one per year, each with a list of missing prize categories for that year.

Remember to use field paths (precede field names with "$") to extract field values in expressions.

In [29]:
# Previous try
original_categories = set(db.prizes.distinct("category", {"year": "1901"}))
pipeline = [
    {'$match': {'category': {'$in': list(original_categories)}}},
    {'$project': {'year': 1, 'category': 1}},
    {'$sort': OrderedDict([('year', -1)])}   # To preserve the order in the dictionary
]
cursor = db.prizes.aggregate(pipeline)
for key, group in groupby(cursor, key=itemgetter("year")):
    missing = original_categories - {doc["category"] for doc in group}
    if missing:
        print("{year}: {missing}".format(year=key, missing=", ".join(sorted(missing))))

2018: literature
1972: peace
1967: peace
1966: peace
1956: peace
1955: peace
1948: peace
1943: literature, peace
1939: peace
1935: literature
1934: physics
1933: chemistry
1932: peace
1931: physics
1928: peace
1925: medicine
1924: chemistry, peace
1923: peace
1921: medicine
1919: chemistry
1918: literature, medicine, peace
1917: chemistry, medicine
1916: chemistry, medicine, peace, physics
1915: medicine, peace
1914: literature, peace


In [30]:
# Using $group
original_categories = sorted(set(db.prizes.distinct(
    "category", 
    {"year": "1901"}
)))

pipeline = [
    {"$match": {"category": {"$in": original_categories}}},
    {"$project": {"category": 1, 
                  "year": 1}},
    
    # Collect the set of category values for each prize year.
    {"$group": {"_id": '$year', 
                "categories": {"$addToSet": "$category"}}},
    
    # Project categories *not* awarded (i.e., that are missing this year).
    {"$project": {"missing": {"$setDifference": [original_categories, 
                                                 '$categories']}}},
    
    # Only include years with at least one missing category
    {"$match": {"missing.0": {"$exists": True}}},
    
    # Sort in reverse chronological order. 
    # Note that "_id" is a distinct year at this stage.
    {"$sort": OrderedDict([("_id", -1)])},
]

for doc in db.prizes.aggregate(pipeline):
    print("{year}: {missing}".format(
        year=doc["_id"],missing=", ".join(sorted(doc["missing"]))
    ))

2018: literature
1972: peace
1967: peace
1966: peace
1956: peace
1955: peace
1948: peace
1943: literature, peace
1939: peace
1935: literature
1934: physics
1933: chemistry
1932: peace
1931: physics
1928: peace
1925: medicine
1924: chemistry, peace
1923: peace
1921: medicine
1919: chemistry
1918: literature, medicine, peace
1917: chemistry, medicine
1916: chemistry, medicine, peace, physics
1915: medicine, peace
1914: literature, peace


In [31]:
db.prizes.aggregate(pipeline).next()

{'_id': '2018', 'missing': ['literature']}

In [32]:
list(db.prizes.aggregate(pipeline))

[{'_id': '2018', 'missing': ['literature']},
 {'_id': '1972', 'missing': ['peace']},
 {'_id': '1967', 'missing': ['peace']},
 {'_id': '1966', 'missing': ['peace']},
 {'_id': '1956', 'missing': ['peace']},
 {'_id': '1955', 'missing': ['peace']},
 {'_id': '1948', 'missing': ['peace']},
 {'_id': '1943', 'missing': ['literature', 'peace']},
 {'_id': '1939', 'missing': ['peace']},
 {'_id': '1935', 'missing': ['literature']},
 {'_id': '1934', 'missing': ['physics']},
 {'_id': '1933', 'missing': ['chemistry']},
 {'_id': '1932', 'missing': ['peace']},
 {'_id': '1931', 'missing': ['physics']},
 {'_id': '1928', 'missing': ['peace']},
 {'_id': '1925', 'missing': ['medicine']},
 {'_id': '1924', 'missing': ['chemistry', 'peace']},
 {'_id': '1923', 'missing': ['peace']},
 {'_id': '1921', 'missing': ['medicine']},
 {'_id': '1919', 'missing': ['chemistry']},
 {'_id': '1918', 'missing': ['literature', 'medicine', 'peace']},
 {'_id': '1917', 'missing': ['chemistry', 'medicine']},
 {'_id': '1916', 'missi

## 4.3 Zoom into Array Fields

### Sizing and summing

In [33]:
# Sizing
list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"},
                  "year": 1, 
                  "category": 1, 
                  "_id": 0}}
]))[:10]

[{'year': '2018', 'category': 'physics', 'n_laureates': 3},
 {'year': '2018', 'category': 'medicine', 'n_laureates': 2},
 {'year': '2018', 'category': 'economics', 'n_laureates': 2},
 {'year': '2017', 'category': 'peace', 'n_laureates': 1},
 {'year': '2017', 'category': 'economics', 'n_laureates': 1},
 {'year': '2016', 'category': 'chemistry', 'n_laureates': 3},
 {'year': '2016', 'category': 'literature', 'n_laureates': 1},
 {'year': '2016', 'category': 'peace', 'n_laureates': 1},
 {'year': '2016', 'category': 'economics', 'n_laureates': 2},
 {'year': '2015', 'category': 'physics', 'n_laureates': 2}]

In [34]:
# Summing
list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"},
                  "category": 1}},
    {"$group": {"_id": "$category", 
                "n_laureates": {"$sum": "$n_laureates"}}},
    {"$sort": {"n_laureates": -1}},
]))

[{'_id': 'medicine', 'n_laureates': 216},
 {'_id': 'physics', 'n_laureates': 210},
 {'_id': 'chemistry', 'n_laureates': 181},
 {'_id': 'peace', 'n_laureates': 133},
 {'_id': 'literature', 'n_laureates': 114},
 {'_id': 'economics', 'n_laureates': 81}]

### How to $unwind
Deconstructs an array field from the input documents to output a document for each element. 

In [35]:
# without $unwind
pprint(list(db.prizes.aggregate([
    {"$project": {"_id": 0, 
                  "year": 1, 
                  "category": 1,
                  "laureates.surname": 1, 
                  "laureates.share": 1}},
    {"$limit": 3}
])))

[{'category': 'physics',
  'laureates': [{'share': '2', 'surname': 'Ashkin'},
                {'share': '4', 'surname': 'Mourou'},
                {'share': '4', 'surname': 'Strickland'}],
  'year': '2018'},
 {'category': 'medicine',
  'laureates': [{'share': '2', 'surname': 'Allison'},
                {'share': '2', 'surname': 'Honjo'}],
  'year': '2018'},
 {'category': 'economics',
  'laureates': [{'share': '2', 'surname': 'Nordhaus'},
                {'share': '2', 'surname': 'Romer'}],
  'year': '2018'}]


In [36]:
# with $unwind
pprint(list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$project": {"_id": 0, 
                  "year": 1, 
                  "category": 1,
                  "laureates.surname": 1, 
                  "laureates.share": 1}},
    {"$limit": 3}
])))

[{'category': 'physics',
  'laureates': {'share': '2', 'surname': 'Ashkin'},
  'year': '2018'},
 {'category': 'physics',
  'laureates': {'share': '4', 'surname': 'Mourou'},
  'year': '2018'},
 {'category': 'physics',
  'laureates': {'share': '4', 'surname': 'Strickland'},
  'year': '2018'}]


### Renormalization, anyone?

In [37]:
# Without $unwind
pprint(list(db.prizes.aggregate([
    {"$project": {"year": 1, 
                  "category": 1, 
                  "laureates.id": 1}},
    {"$group": {"_id": {"$concat": ["$category", ":", "$year"]},
                "laureate_ids": {"$addToSet": "$laureates.id"}}},
    {"$limit": 5}
])))

[{'_id': 'medicine:1968', 'laureate_ids': [['388', '389', '390']]},
 {'_id': 'medicine:1981', 'laureate_ids': [['422', '423', '424']]},
 {'_id': 'physics:1973', 'laureate_ids': [['97', '98', '99']]},
 {'_id': 'medicine:1971', 'laureate_ids': [['397']]},
 {'_id': 'peace:1901', 'laureate_ids': [['462', '463']]}]


In [38]:
# With $unwind
pprint(list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$project": {"year": 1, 
                  "category": 1, 
                  "laureates.id": 1}},
    {"$group": {"_id": {"$concat": ["$category", ":", "$year"]},
                "laureate_ids": {"$addToSet": "$laureates.id"}}},
    {"$limit": 5}
])))

[{'_id': 'peace:1917', 'laureate_ids': ['482']},
 {'_id': 'medicine:1986', 'laureate_ids': ['435', '434']},
 {'_id': 'medicine:1911', 'laureate_ids': ['305']},
 {'_id': 'economics:1986', 'laureate_ids': ['700']},
 {'_id': 'literature:1955', 'laureate_ids': ['626']}]


### $unwind and count 'em, one by one

In [39]:
list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"}, 
                  "category": 1}},
    {"$group": {"_id": "$category", 
                "n_laureates": {"$sum": "$n_laureates"}}},
    {"$sort": {"n_laureates": -1}},
]))

[{'_id': 'medicine', 'n_laureates': 216},
 {'_id': 'physics', 'n_laureates': 210},
 {'_id': 'chemistry', 'n_laureates': 181},
 {'_id': 'peace', 'n_laureates': 133},
 {'_id': 'literature', 'n_laureates': 114},
 {'_id': 'economics', 'n_laureates': 81}]

In [40]:
list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$group": {"_id": "$category", "n_laureates": {"$sum": 1}}},
    {"$sort": {"n_laureates": -1}},
]))

[{'_id': 'medicine', 'n_laureates': 216},
 {'_id': 'physics', 'n_laureates': 210},
 {'_id': 'chemistry', 'n_laureates': 181},
 {'_id': 'peace', 'n_laureates': 133},
 {'_id': 'literature', 'n_laureates': 114},
 {'_id': 'economics', 'n_laureates': 81}]

### $lookup

In [41]:
# Reviewing prizes
pprint(db.prizes.find_one({}))

{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'category': 'physics',
 'laureates': [{'firstname': 'Arthur',
                'id': '960',
                'motivation': '"for the optical tweezers and their application '
                              'to biological systems"',
                'share': '2',
                'surname': 'Ashkin'},
               {'firstname': 'Gérard',
                'id': '961',
                'motivation': '"for their method of generating high-intensity, '
                              'ultra-short optical pulses"',
                'share': '4',
                'surname': 'Mourou'},
               {'firstname': 'Donna',
                'id': '962',
                'motivation': '"for their method of generating high-intensity, '
                              'ultra-short optical pulses"',
                'share': '4',
                'surname': 'Strickland'}],
 'overallMotivation': '“for groundbreaking inventions in the field of laser '
                   

In [42]:
# Reviewing laureates
pprint(db.laureates.find_one({}))

{'_id': ObjectId('6706d88371ea025ecc350f6e'),
 'born': '1853-07-18',
 'bornCity': 'Arnhem',
 'bornCountry': 'the Netherlands',
 'bornCountryCode': 'NL',
 'died': '1928-02-04',
 'diedCountry': 'the Netherlands',
 'diedCountryCode': 'NL',
 'firstname': 'Hendrik Antoon',
 'gender': 'male',
 'id': '2',
 'prizes': [{'affiliations': [{'city': 'Leiden',
                               'country': 'the Netherlands',
                               'name': 'Leiden University'}],
             'category': 'physics',
             'motivation': '"in recognition of the extraordinary service they '
                           'rendered by their researches into the influence of '
                           'magnetism upon radiation phenomena"',
             'share': '2',
             'year': '1902'}],
 'surname': 'Lorentz'}


In [43]:
born1 = list(db.prizes.aggregate([
    {"$match": {"category": "economics"}},
    {"$unwind": "$laureates"},
    {"$lookup": {"from": "laureates", 
                 "foreignField": "id",
                 "localField": "laureates.id", 
                 "as": "laureate_bios"}},
    {"$unwind": "$laureate_bios"},
    {"$group": {"_id": None,
                "bornCountries":{"$addToSet": "$laureate_bios.bornCountry"}}},
]))
pprint(born1)

[{'_id': None,
  'bornCountries': ['Russian Empire (now Russia)',
                    'France',
                    'Russia',
                    'Germany (now Poland)',
                    'Russian Empire (now Belarus)',
                    'the Netherlands',
                    'British West Indies (now Saint Lucia)',
                    'Hungary',
                    'Norway',
                    'USA',
                    'Scotland',
                    'Canada',
                    'India',
                    'Germany',
                    'Sweden',
                    'Cyprus',
                    'United Kingdom',
                    'British Mandate of Palestine (now Israel)',
                    'Austria',
                    'Finland',
                    'Italy']}]


In [44]:
born2 = db.laureates.distinct(
    "bornCountry", {"prizes.category": "economics"}
)
pprint(born2)
assert set(born2) == set(born1[0]['bornCountries'])

['Austria',
 'British Mandate of Palestine (now Israel)',
 'British West Indies (now Saint Lucia)',
 'Canada',
 'Cyprus',
 'Finland',
 'France',
 'Germany',
 'Germany (now Poland)',
 'Hungary',
 'India',
 'Italy',
 'Norway',
 'Russia',
 'Russian Empire (now Belarus)',
 'Russian Empire (now Russia)',
 'Scotland',
 'Sweden',
 'USA',
 'United Kingdom',
 'the Netherlands']


### Excercises

#### Embedding aggregation expressions

The `$expr` operator allows embedding of aggregation expressions in a normal query (or in a `$match` stage).

**How to counts the number of laureate documents with string-valued bornCountries when passed to db.laureates.count_documents?**

In [45]:
db.laureates.count_documents({})

934

In [46]:
db.laureates.count_documents({
    "bornCountry": {"$in": db.laureates.distinct("bornCountry")},
    "bornCountry": {'$exists': True}  # Excluding org
})

901

In [47]:
db.laureates.count_documents({
    "$expr": {"$in": ["$bornCountry",   # org doesnot have bornCountry
                      db.laureates.distinct("bornCountry")]}
})

901

In [48]:
db.laureates.count_documents({ # org doesnot have bornCountry
    "$expr": {"$eq": [{"$type": "$bornCountry"}, "string"]}
})

901

In [49]:
db.laureates.count_documents({
    "bornCountry": {"$type": "string"}
})

901

#### Here and elsewhere
What proportion of laureates won a prize while affiliated with an institution in their country of birth? Build an aggregation pipeline to get the count of laureates who either did or did not win a prize with an affiliation country that is a substring of their country of birth -- for example, the prize affiliation country "Germany" should match the country of birth "Prussia (now Germany)".

In [50]:
key_ac = "prizes.affiliations.country"
key_bc = "bornCountry"

pipeline = [
    {"$project": {key_bc: 1, key_ac: 1}},

    # Ensure a single prize affiliation country per pipeline document
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},

    # Ensure values in the list of distinct values (so not empty)
    {"$match": {key_ac: {'$in': db.laureates.distinct(key_ac)}}},
    {"$project": {
        "affilCountrySameAsBorn": {
            "$gte": [
                {"$indexOfBytes": ["$"+key_ac, "$"+key_bc]}, 
                0
            ]
        }
    }},

    # Count by "$affilCountrySameAsBorn" value (True or False)
    {"$group": {"_id": "$affilCountrySameAsBorn",
                "count": {"$sum": 1}}},
]

list(db.laureates.aggregate(pipeline))

[{'_id': False, 'count': 261}, {'_id': True, 'count': 477}]

In [51]:
# Building step by step
# 1 - projection
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {"$limit": 3}
])))

[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'bornCountry': 'the Netherlands',
  'prizes': [{'affiliations': [{'country': 'the Netherlands'}]}]},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'bornCountry': 'USA',
  'prizes': [{'affiliations': [{'country': 'USA'}]}]},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'bornCountry': 'USA',
  'prizes': [{'affiliations': [{'country': 'USA'}]}]}]


In [52]:
# Building step by step
# 2. $unwind
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {"$limit": 3}
])))
print('-----------------------------------')
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},
    {"$limit": 3}
])))

[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'bornCountry': 'the Netherlands',
  'prizes': {'affiliations': [{'country': 'the Netherlands'}]}},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': [{'country': 'USA'}]}},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': [{'country': 'USA'}]}}]
-----------------------------------
[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'bornCountry': 'the Netherlands',
  'prizes': {'affiliations': {'country': 'the Netherlands'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': {'country': 'USA'}}}]


In [53]:
# Building step by step
# 3. $match
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},
    {"$match": {key_ac: {'$in': db.laureates.distinct(key_ac)}}},
    {"$limit": 3}
])))

[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'bornCountry': 'the Netherlands',
  'prizes': {'affiliations': {'country': 'the Netherlands'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'bornCountry': 'USA',
  'prizes': {'affiliations': {'country': 'USA'}}}]


In [54]:
# Building step by step
# 4. $indexOfBytes
# Searches a string for an occurrence of a substring and 
# returns the UTF-8 byte index (zero-based) of the first occurrence. 
# If the substring is not found, returns -1.
# $indexOfBytes has the following operator expression syntax:
# { $indexOfBytes: [ <string expression>, <substring expression>, 
#                    <start>, <end> ] }
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},
    {"$match": {key_ac: {'$in': db.laureates.distinct(key_ac)}}},
    {"$project": {'bornCountry': 1,
                  'prizes': 1,
                  'idx_of_bytes_value': {"$indexOfBytes": ["$"+key_ac, 
                                                           "$"+key_bc]}}},
    {"$limit": 8}
])))

[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'bornCountry': 'the Netherlands',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'the Netherlands'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'bornCountry': 'USA',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'bornCountry': 'USA',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f71'),
  'bornCountry': 'United Kingdom',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'United Kingdom'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f72'),
  'bornCountry': 'United Kingdom',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'United Kingdom'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f73'),
  'bornCountry': 'United Kingdom',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'United Kingdom'}}},
 {'_id': ObjectI

In [55]:
# Building step by step
# 5. complete the $project
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},
    {"$match": {key_ac: {'$in': db.laureates.distinct(key_ac)}}},
    {"$project": {'bornCountry': 1,
                  'prizes': 1,
                  'idx_of_bytes_value': {"$indexOfBytes": ["$"+key_ac, 
                                                           "$"+key_bc]},
                  'affilCountrySameAsBorn': {
                      "$gte": [
                          {"$indexOfBytes": ["$"+key_ac, "$"+key_bc]},
                          0
                      ]
                  }}},
    {"$limit": 8}
])))

[{'_id': ObjectId('6706d88371ea025ecc350f6e'),
  'affilCountrySameAsBorn': True,
  'bornCountry': 'the Netherlands',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'the Netherlands'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f6f'),
  'affilCountrySameAsBorn': True,
  'bornCountry': 'USA',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f70'),
  'affilCountrySameAsBorn': True,
  'bornCountry': 'USA',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'USA'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f71'),
  'affilCountrySameAsBorn': True,
  'bornCountry': 'United Kingdom',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'United Kingdom'}}},
 {'_id': ObjectId('6706d88371ea025ecc350f72'),
  'affilCountrySameAsBorn': True,
  'bornCountry': 'United Kingdom',
  'idx_of_bytes_value': 0,
  'prizes': {'affiliations': {'country': 'United Kingdom'}}},
 {'_id': ObjectId

In [56]:
# Building step by step
# 6. $group
pprint(list(db.laureates.aggregate([
    {"$project": {key_bc: 1, key_ac: 1}},
    {'$unwind': "$prizes"},
    {'$unwind': "$prizes.affiliations"},
    {"$match": {key_ac: {'$in': db.laureates.distinct(key_ac)}}},
    {"$project": {'bornCountry': 1,
                  'prizes': 1,
                  'idx_of_bytes_value': {"$indexOfBytes": ["$"+key_ac, 
                                                           "$"+key_bc]},
                  'affilCountrySameAsBorn': {
                      "$gte": [
                          {"$indexOfBytes": ["$"+key_ac, "$"+key_bc]},
                          0
                      ]
                  }}},
    {"$group": {"_id": "$affilCountrySameAsBorn",
                "count": {"$sum": 1}}},
    {"$limit": 8}
])))

[{'_id': False, 'count': 261}, {'_id': True, 'count': 477}]


#### Countries of birth by prize category

Some prize categories have laureates hailing from a greater number of countries than do other categories. You will build an aggregation pipeline for the prizes collection to collect these numbers, using a $lookup stage to obtain laureate countries of birth.

In [57]:
pipeline = [
    # Unwind the laureates array
    {'$unwind': "$laureates"},
    {"$lookup": {
        "from": "laureates", "foreignField": "id",
        "localField": "laureates.id", "as": "laureate_bios"}},

    # Unwind the new laureate_bios array
    {"$unwind": '$laureate_bios'},
    {"$project": {"category": 1,
                  "bornCountry": "$laureate_bios.bornCountry"}},  # renaming

    # Collect bornCountry values associated with each prize category
    {"$group": {'_id': "$category",
                "bornCountries": {"$addToSet": "$bornCountry"}}},

    # Project out the size of each category's (set of) bornCountries
    {"$project": {"category": 1,
                  "nBornCountries": {"$size": '$bornCountries'}}},
    {"$sort": {"nBornCountries": -1}},
]

pprint(list(db.prizes.aggregate(pipeline)))

[{'_id': 'literature', 'nBornCountries': 55},
 {'_id': 'peace', 'nBornCountries': 50},
 {'_id': 'chemistry', 'nBornCountries': 48},
 {'_id': 'medicine', 'nBornCountries': 44},
 {'_id': 'physics', 'nBornCountries': 44},
 {'_id': 'economics', 'nBornCountries': 21}]


In [58]:
# Taking a look into $lookup
pipeline = [
    {'$unwind': "$laureates"},
    {"$lookup": {
        "from": "laureates", "foreignField": "id",
        "localField": "laureates.id", "as": "laureate_bios"}},
]

pprint(list(db.prizes.aggregate(pipeline))[0])

{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'category': 'physics',
 'laureate_bios': [{'_id': ObjectId('6706d88371ea025ecc351312'),
                    'born': '1922-09-02',
                    'bornCity': 'New York, NY',
                    'bornCountry': 'USA',
                    'bornCountryCode': 'US',
                    'died': '0000-00-00',
                    'firstname': 'Arthur',
                    'gender': 'male',
                    'id': '960',
                    'prizes': [{'affiliations': [{'city': 'Holmdel, NJ',
                                                  'country': 'USA',
                                                  'name': 'Bell Laboratories'}],
                                'category': 'physics',
                                'motivation': '"for the optical tweezers and '
                                              'their application to biological '
                                              'systems"',
                                'over

In [59]:
# Adding the $unwind stage
pipeline = [
    {'$unwind': "$laureates"},
    {"$lookup": {
        "from": "laureates", "foreignField": "id",
        "localField": "laureates.id", "as": "laureate_bios"}},
    {"$unwind": '$laureate_bios'},
]

pprint(list(db.prizes.aggregate(pipeline))[0])

{'_id': ObjectId('6706d88371ea025ecc350d20'),
 'category': 'physics',
 'laureate_bios': {'_id': ObjectId('6706d88371ea025ecc351312'),
                   'born': '1922-09-02',
                   'bornCity': 'New York, NY',
                   'bornCountry': 'USA',
                   'bornCountryCode': 'US',
                   'died': '0000-00-00',
                   'firstname': 'Arthur',
                   'gender': 'male',
                   'id': '960',
                   'prizes': [{'affiliations': [{'city': 'Holmdel, NJ',
                                                 'country': 'USA',
                                                 'name': 'Bell Laboratories'}],
                               'category': 'physics',
                               'motivation': '"for the optical tweezers and '
                                             'their application to biological '
                                             'systems"',
                               'overallMotivation': '

## 4.4 Something Extra: $addFields to Aid Analysis

### A somber $project

In [60]:
# Reviewing laureates
db.laureates.find_one({})

{'_id': ObjectId('6706d88371ea025ecc350f6e'),
 'id': '2',
 'firstname': 'Hendrik Antoon',
 'surname': 'Lorentz',
 'born': '1853-07-18',
 'died': '1928-02-04',
 'bornCountry': 'the Netherlands',
 'bornCountryCode': 'NL',
 'bornCity': 'Arnhem',
 'diedCountry': 'the Netherlands',
 'diedCountryCode': 'NL',
 'gender': 'male',
 'prizes': [{'year': '1902',
   'category': 'physics',
   'share': '2',
   'motivation': '"in recognition of the extraordinary service they rendered by their researches into the influence of magnetism upon radiation phenomena"',
   'affiliations': [{'name': 'Leiden University',
     'city': 'Leiden',
     'country': 'the Netherlands'}]}]}

In [61]:
%%expect_exception OperationFailure   

# Is there any missing died and born fields?
docs = list(db.laureates.aggregate([
    {"$project": {
        "died": {"$dateFromString": {"dateString": "$died"}},
        "born": {"$dateFromString": {"dateString": "$born"}}
    }}
]))

---------------------------------------------------------------------------
OperationFailure                          Traceback (most recent call last)
Cell In[61], line 2
      1 # Is there any missing died and born fields?
----> 2 docs = list(db.laureates.aggregate([
      3     {"$project": {
      4         "died": {"$dateFromString": {"dateString": "$died"}},
      5         "born": {"$dateFromString": {"dateString": "$born"}}
      6     }}
      7 ]))

File C:\ProgramData\anaconda3\envs\mongo\Lib\site-packages\pymongo\collection.py:2696, in Collection.aggregate(self, pipeline, session, let, comment, **kwargs)
   2620 """Perform an aggregation using the aggregation framework on this
   2621 collection.
   2622 
   (...)
   2693     https://mongodb.com/docs/manual/reference/command/aggregate
   2694 """
   2695 with self.__database.client._tmp_session(session, close=False) as s:
-> 2696     return self._aggregate(
   2697         _CollectionAggregationCommand,
   2698         pipe

In [62]:
# There is no missing died and born fields
db.laureates.count_documents({
    '$or': [{'died': {'$exists': False}},
            {'born': {'$exists': False}}]
})

0

In [63]:
# The Error says parsing date string '0000-00-00'
print('Total docs:', db.laureates.count_documents({}))
db.laureates.count_documents({
    '$or': [{'died': {'$eq': '0000-00-00'}},
            {'born': {'$eq': '0000-00-00'}}]
})

Total docs: 934


327

In [64]:
%%expect_exception OperationFailure   

# Filtering dates greater than 1700
# New error: Error parsing date string '1898-00-00'
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, 
                "born": {"$gt": "1700"}}},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                  "born": {"$dateFromString": {"dateString": "$born"}}}}
]))

---------------------------------------------------------------------------
OperationFailure                          Traceback (most recent call last)
Cell In[64], line 3
      1 # Filtering dates greater than 1700
      2 # New error: Error parsing date string '1898-00-00'
----> 3 docs = list(db.laureates.aggregate([
      4     {"$match": {"died": {"$gt": "1700"}, 
      5                 "born": {"$gt": "1700"}}},
      6     {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
      7                   "born": {"$dateFromString": {"dateString": "$born"}}}}
      8 ]))

File C:\ProgramData\anaconda3\envs\mongo\Lib\site-packages\pymongo\command_cursor.py:309, in CommandCursor.next(self)
    307 # Block until a document is returnable.
    308 while self.alive:
--> 309     doc = self._try_next(True)
    310     if doc is not None:
    311         return doc

File C:\ProgramData\anaconda3\envs\mongo\Lib\site-packages\pymongo\command_cursor.py:320, in CommandCursor._try_n

### split and cond-itionally correct (with $concat)

In [65]:
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, "born": {"$gt": "1700"}}},
    {"$addFields": {"bornArray": {"$split": ["$born", "-"]},
                    "diedArray": {"$split": ["$died", "-"]}}},
    {"$addFields": {
        "born": {"$cond": [{"$in": ["00", "$bornArray"]},  # condition
                           {"$concat": [{"$arrayElemAt": ["$bornArray", 0]}, 
                                        "-01-01"]},  # if True
                           "$born"]}   # if false
    }},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                  "born": {"$dateFromString": {"dateString": "$born"}},
                  "_id": 0}}
]))
pprint(docs[:5])

[{'born': datetime.datetime(1853, 7, 18, 0, 0),
  'died': datetime.datetime(1928, 2, 4, 0, 0)},
 {'born': datetime.datetime(1918, 9, 27, 0, 0),
  'died': datetime.datetime(1984, 10, 14, 0, 0)},
 {'born': datetime.datetime(1922, 6, 19, 0, 0),
  'died': datetime.datetime(2009, 9, 8, 0, 0)},
 {'born': datetime.datetime(1917, 12, 9, 0, 0),
  'died': datetime.datetime(1986, 3, 31, 0, 0)},
 {'born': datetime.datetime(1931, 3, 22, 0, 0),
  'died': datetime.datetime(2018, 7, 18, 0, 0)}]


### A $bucket list
Categorizes incoming documents into groups, called buckets, based on a specified expression and bucket boundaries and outputs a document per each bucket.

In [66]:
# Calculating the age
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, "born": {"$gt": "1700"}}},
    {"$addFields": {"bornArray": {"$split": ["$born", "-"]},
                    "diedArray": {"$split": ["$died", "-"]}}},
    {"$addFields": {
        "born": {"$cond": [{"$in": ["00", "$bornArray"]},  # condition
                           {"$concat": [{"$arrayElemAt": ["$bornArray", 0]}, 
                                        "-01-01"]},  # if True
                           "$born"]}   # if false
    }},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                  "born": {"$dateFromString": {"dateString": "$born"}},
                  "_id": 0,
    }},
    {"$project": {
        "died": 1,
        "born": 1,
        "years": {
            "$floor": {
                "$divide": [
                    {"$subtract": ["$died", "$born"]},
                    31557600000 # 1000 * 60 * 60 * 24 * 365.25
                ]
            }
        }
    }}
]))

pprint(docs[:5])

[{'born': datetime.datetime(1853, 7, 18, 0, 0),
  'died': datetime.datetime(1928, 2, 4, 0, 0),
  'years': 74.0},
 {'born': datetime.datetime(1918, 9, 27, 0, 0),
  'died': datetime.datetime(1984, 10, 14, 0, 0),
  'years': 66.0},
 {'born': datetime.datetime(1922, 6, 19, 0, 0),
  'died': datetime.datetime(2009, 9, 8, 0, 0),
  'years': 87.0},
 {'born': datetime.datetime(1917, 12, 9, 0, 0),
  'died': datetime.datetime(1986, 3, 31, 0, 0),
  'years': 68.0},
 {'born': datetime.datetime(1931, 3, 22, 0, 0),
  'died': datetime.datetime(2018, 7, 18, 0, 0),
  'years': 87.0}]


In [67]:
# Adding $bucket
# Calculating the age
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, "born": {"$gt": "1700"}}},
    {"$addFields": {"bornArray": {"$split": ["$born", "-"]},
                    "diedArray": {"$split": ["$died", "-"]}}},
    {"$addFields": {
        "born": {"$cond": [{"$in": ["00", "$bornArray"]},  # condition
                           {"$concat": [{"$arrayElemAt": ["$bornArray", 0]}, 
                                        "-01-01"]},  # if True
                           "$born"]}   # if false
    }},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                  "born": {"$dateFromString": {"dateString": "$born"}},
                  "_id": 0,
    }},
    {"$project": {
        "died": 1,
        "born": 1,
        "years": {
            "$floor": {
                "$divide": [
                    {"$subtract": ["$died", "$born"]},
                    31557600000 # 1000 * 60 * 60 * 24 * 365.25
                ]
            }
        }
    }},
    {"$bucket": {"groupBy": "$years",
                 "boundaries": list(range(30, 120, 10))}}
]))

pprint(docs)

[{'_id': 30, 'count': 1},
 {'_id': 40, 'count': 6},
 {'_id': 50, 'count': 21},
 {'_id': 60, 'count': 87},
 {'_id': 70, 'count': 154},
 {'_id': 80, 'count': 221},
 {'_id': 90, 'count': 115},
 {'_id': 100, 'count': 2}]


### Excercises
#### ...it's the life in your years

For the pipeline we developed in the last lesson, We want you to replace the last ($bucket) stage to get the max age and min age.

In [68]:
# Calculating the age
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, "born": {"$gt": "1700"}}},
    {"$addFields": {"bornArray": {"$split": ["$born", "-"]},
                    "diedArray": {"$split": ["$died", "-"]}}},
    {"$addFields": {
        "born": {"$cond": [{"$in": ["00", "$bornArray"]},  # condition
                           {"$concat": [{"$arrayElemAt": ["$bornArray", 0]}, 
                                        "-01-01"]},  # if True
                           "$born"]}   # if false
    }},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                  "born": {"$dateFromString": {"dateString": "$born"}},
                  "_id": 0,
    }},
    {"$project": {
        "died": 1,
        "born": 1,
        "years": {
            "$floor": {
                "$divide": [
                    {"$subtract": ["$died", "$born"]},
                    31557600000 # 1000 * 60 * 60 * 24 * 365.25
                ]
            }
        }
    }},
    {"$project": {"years": 1, 
                  "firstname": 1, 
                  "surname": 1, 
                  "_id": 0}}
]))

In [69]:
print(max(docs, key=itemgetter("years")))
print(min(docs, key=itemgetter("years")))

{'years': 103.0}
{'years': 39.0}


#### How many prizes were awarded to immigrants?
How many prizes were awarded to people who had no affiliation in their country of birth at the time of the award?

In [70]:
pipeline = [
    # Limit results to people; project needed fields; unwind prizes
    {'$match': {'gender': {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
  
    # Count prizes with no country-of-birth affiliation
    {"$addFields": {
        "bornCountryInAffiliations": {
            "$in": ['$bornCountry', "$prizes.affiliations.country"]
        }
    }},
    {'$match': {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

print(list(db.laureates.aggregate(pipeline)))

[{'awardedElsewhere': 478}]


#### Refinement: filter out "unaffiliated" people
In the previous exercise, we counted prizes awarded to people without an affiliation in their "bornCountry". However, hundreds of prizes were awarded to people without recorded affiliations; sure, their "bornCountry" is technically not the "country" of any of their affiliations, but there are no "country" values to compare against!

- Construct a stage added_stage that filters for laureate "prizes.affiliations.country" values that are non-empty, that is, are $in a list of the distinct values that the field takes in the collection.
- Insert this stage into the pipeline so that it filters out single prizes (not arrays) and precedes any test for membership in an array of countries. Recall that the first parameter to <list>.insert is the (zero-based) index for insertion.

In [71]:
# Checking there is not None
print(db.laureates.distinct('prizes.affiliations.country'))

['Alsace (then Germany, now France)', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Canada', 'China', 'Czechoslovakia', 'Denmark', 'Federal Republic of Germany', 'Finland', 'France', 'Germany', 'Hungary', 'India', 'Ireland', 'Israel', 'Italy', 'Japan', 'Norway', 'Portugal', 'Russia', 'Spain', 'Sweden', 'Switzerland', 'USA', 'USSR', 'United Kingdom', 'the Netherlands']


In [72]:
pipeline = [
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
    {"$addFields": {"bornCountryInAffiliations": {"$in": ["$bornCountry", "$prizes.affiliations.country"]}}},
    {"$match": {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

# Construct the additional filter stage
added_stage = {"$match": {"prizes.affiliations.country": {
    '$in': db.laureates.distinct("prizes.affiliations.country")}}}

# Insert this stage into the pipeline
pipeline.insert(3, added_stage)
print(list(db.laureates.aggregate(pipeline)))

[{'awardedElsewhere': 252}]


-------------------------------------